In [3]:
!pip install crewai crewai_tools python_dotenv

  Using cached crewai-0.159.0-py3-none-any.whl.metadata (35 kB)
  Using cached crewai_tools-0.62.0-py3-none-any.whl.metadata (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.2/379.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.0/677.0 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━

In [25]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
import os
os.environ["OPENAI_MODEL_NAME"]='gpt-4o-mini'

In [10]:
from crewai import Agent, Task, Crew

# AI Agent to support creation of articles

##Creating Agents
Define Agents and provide a role, goal and backstory

In [37]:
planner = Agent(
    role="Content planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
                "about the topic {topic}."
                "You collect information that helps the "
                "audience learn something and make informed decisions "
                "Your work is the basis for the Content Writer to write an article on the topic"
                "Content should be written in italian language",
    allow_delegation=False,
    verbose=True
)

In [38]:
writer = Agent(
    role="Content writer",
    goal="Write insightful and factually accurate opinion piece about the topic {topic}",
    backstory="You're working on a writing a new opinion piece about the topic {topic} "
                "You base your writing on the work of the COntent Planner, who provides an outline "
                "and relevant context about the topic. "
                "You follow the main objectives and direction of the outline, as provided to you by the Content Planner. "
                "You also provide objective and impartial insights and back them up with information provide by Content Planner."
                "You acknowledge in your opinion piece when your statements are opiniions as opposed to objective statements."
                "Article should be written in italian language",
    allow_delegation=False,
    verbose=True
)

In [29]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with the writing style of the organization",
    backstory="You are an editor who receives a blog post from the COntent Writer. "
                "Your goal is to review the blog post to ensure that is follows journalistic best practices, "
                "provide balanced viewpoints when providing opinions or asssertions, "
                "and also avoid major controversial topics or opinions when possibile "
                "Article should be written in italian langue",
    allow_delegation=False,
    verbose=True
)

##Defining Tasks
Defining tasks and provide them a description, expected_output and agent

In [20]:
plan = Task(
    description=(
        "1. Prioritize latest trends on topic {topic}\n"
        "2. Identify the target audience considering their interests.\n"
        "3. Develop a detailed content outline including intro, key points and a call to action.\n"
        "4. Include SEO keywords and relevant sources"
    ),
    expected_output="A comprehensive content plan document woth outline, audience analysis, SEO keywords and resources",
    agent=planner
)

In [21]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling blog post on {topic}\n"
        "2. Incorporate SEO keywords naturally\n"
        "3. Ensure the post is structured with an engagin intro, insightful body and summarizing conclusion."
    ),
    expected_output="A well written blog post in markdow format ready for publication. Each section should have 2 or 3 paragraphs.",
    agent=writer
)

In [33]:
edit = Task(
    description=(
        "Review the given blog post for grammatical errors and alignment with the brand's voice that is a journal for teenagers."
    ),
    expected_output="A well written blog post in markdow format ready for publication. Each section should have 2 or 3 paragraphs.",
    agent=editor
)

##Creating the Crew
Create crew of agents and pass them the tasks.

In [23]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

In [34]:
result = crew.kickoff(inputs={"topic": "Stipendio dei giocatori di calcio"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 39f2c808-b195-403f-9757-9472daa6d3e8                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: 1. Prioritize latest trends on topic Stipendio dei giocatori di calcio                                   │
│  2. Identify the target audience considering their interests.                                                   │
│  3. Develop a detailed content outline including intro, key points and a call to action.                        │
│  4. Include SEO keywords and relevant sources                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Comprehensive Content Plan Document: Stipendio dei Giocatori di Calcio**                                     │
│                                                                                                                 │
│  **1. Target Audience Analysis:**                                                                               │
│     - **Demographics:** Primarily young adults aged 18-35, with a strong interest in football (soccer).         │
│     - **Interests:** Fans of football, sports analysts, aspiring football players, and individuals interested   │
│  in sports economics.                                                                                           │
│     - **Goals:** The audience seeks to understand current trends in football salaries, implications on the      │
│  sport, and how it affects player performance and team dynamics.                                                │
│                                                                                                                 │
│  **2. Content Outline:**                                                                                        │
│                                                                                                                 │
│     **Title:** "L’Impennata dei Stipendi nel Calcio: Cosa Significa per i Giocatori e i Club?"                  │
│                                                                                                                 │
│     **Introduction:**                                                                                           │
│     - Brief overview of the current trend in football player salaries.                                          │
│     - Importance of understanding salaries in the context of player performance and market dynamics.            │
│                                                                                                                 │
│     **Key Points:**                                                                                             │
│     - **1. Recent Trends in Football Salaries:**                                                                │
│       - Overview of average player salaries across major leagues (Serie A, Premier League, La Liga).            │
│       - Discussion on the rise in wages and what factors contribute to this increase (e.g., broadcasting        │
│  rights, sponsorships).                                                                                         │
│                                                                                                                 │
│     - **2. Impact of High Salaries on Player Performance:**                                                     │
│       - Analysis of how salaries affect player motivation and team dynamics.                                    │
│       - Examples of players whose salaries have influenced their performance (positive and negative).           │
│                                                                                                                 │
│     - **3. Comparisons between Leagues:**                                                                       │
│       - Contrast football player salaries in different countries and leagues.                                   │
│       - Insights on how financial disparities affect pl

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d5a5ae7e-b154-4bb1-87d0-9ed77d418a03                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Review the given blog post for grammatical errors and alignment with the brand's voice.                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # L’Impennata dei Stipendi nel Calcio: Cosa Significa per i Giocatori e i Club?                                │
│                                                                                                                 │
│  ## Introduzione                                                                                                │
│  Negli ultimi anni, il calcio ha visto una continua e impressionante impennata dei stipendi dei giocatori, con  │
│  cifre che adesso sembrano quasi extraterrestri. Questa crescita non è solo una questione di numeri; è          │
│  fondamentale comprendere come queste retribuzioni influenzino le performance individuali degli atleti e la     │
│  dinamica delle squadre. In questo articolo, esploreremo i trend attuali nei stipendi dei calciatori, come      │
│  influiscono sulla motivazione e sul morale dei giocatori, e cosa possiamo aspettarci in futuro in questa       │
│  economia sportiva in continua evoluzione.                                                                      │
│                                                                                                                 │
│  Il tema degli stipendi nel calcio è di particolare rilevanza, non solo per i tifosi, ma anche per coloro che   │
│  sono coinvolti nelle decisioni strategiche dei club. Comprendere la situazione salariale attuale aiuta non     │
│  solo a contestualizzare le cifre, ma anche a valutare le implicazioni più ampie sui club e sulle rispettive    │
│  performance. I salari dei calciatori, infatti, possiedono un forte impatto non solo sulla loro carriera, ma    │
│  anche sulla visibilità e sulla competitività di una lega.                                                      │
│                                                                                                                 │
│  ## 1. Recent Trends in Football Salaries                                                                       │
│  Iniziamo analizzando i salari medi dei calciatori nelle leghe maggiori come la Serie A, la Premier League e    │
│  La Liga. Ad esempio, la Premier League continua a dominare con stipendi medi di oltre 3 milioni di euro        │
│  all’anno, mentre la Serie A e La Liga stanno gradualmente recuperando terreno. Questo aumento esponenziale è   │
│  in gran parte attribuibile a un’esplosione dei diritti televisivi e degli sponsor, che hanno ricolmato le      │
│  casse dei club con liquidità significativa.                                                                    │
│                                                                                                                 │
│  Oltre ai diritti di trasmissione, l'interesse globale per il calcio ha portato a una crescita senza            │
│  precedenti nel merchandising e nelle vendite di biglietti, contribuendo a un circolo virtuoso di investimenti  │
│  e guadagni. Queste forze di mercato hanno reso i giocatori d'élite più desiderabili, giustificando le cifre    │
│  stratosferiche che vediamo oggi nei loro contratti. Questo fenomeno ha creato anche un’aspettativa tra i       │
│  giocatori stessi riguardo alla retribuzione, imponendo ai club di essere competitivi non solo in campo, ma     │
│  anche sul fronte economico.                           

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7fe28b69-9b86-4b27-aa4b-67113b1a2283                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 39f2c808-b195-403f-9757-9472daa6d3e8                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # L’Impennata dei Stipendi nel Calcio: Cosa Significa per i Giocatori e i Club?                                │
│                                                                                                                 │
│  ## Introduzione                                                                                                │
│  Negli ultimi anni, il calcio ha visto una continua e impressionante impennata dei stipendi dei giocatori, con  │
│  cifre che adesso sembrano quasi extraterrestri. Questa crescita non è solo una questione di numeri; è          │
│  fondamentale comprendere come queste retribuzioni influenzino le performance individuali degli atleti e la     │
│  dinamica delle squadre. In questo articolo, esploreremo i trend attuali nei stipendi dei calciatori, come      │
│  influiscono sulla motivazione e sul morale dei giocatori, e cosa possiamo aspettarci in futuro in questa       │
│  economia sportiva in continua evoluzione.                                                                      │
│                                                                                                                 │
│  Il tema degli stipendi nel calcio è di particolare rilevanza, non solo per i tifosi, ma anche per coloro che   │
│  sono coinvolti nelle decisioni strategiche dei club. Comprendere la situazione salariale attuale aiuta non     │
│  solo a contestualizzare le cifre, ma anche a valutare le implicazioni più ampie sui club e sulle rispettive    │
│  performance. I salari dei calciatori, infatti, possiedono un forte impatto non solo sulla loro carriera, ma    │
│  anche sulla visibilità e sulla competitività di una lega.                                                      │
│                                                                                                                 │
│  ## 1. Recent Trends in Football Salaries                                                                       │
│  Iniziamo analizzando i salari medi dei calciatori nelle leghe maggiori come la Serie A, la Premier League e    │
│  La Liga. Ad esempio, la Premier League continua a dominare con stipendi medi di oltre 3 milioni di euro        │
│  all’anno, mentre la Serie A e La Liga stanno gradualmente recuperando terreno. Questo aumento esponenziale è   │
│  in gran parte attribuibile a un’esplosione dei diritti televisivi e degli sponsor, che hanno ricolmato le      │
│  casse dei club con liquidità significativa.                                                                    │
│                                                                                                                 │
│  Oltre ai diritti di trasmissione, l'interesse globale per il calcio ha portato a una crescita senza            │
│  precedenti nel merchandising e nelle vendite di biglietti, contribuendo a un circolo virtuoso di investimenti  │
│  e guadagni. Queste forze di mercato hanno reso i giocatori d'élite più desiderabili, giustificando le cifre    │
│  stratosferiche che vediamo oggi nei loro contratti. Questo fenomeno ha creato anche un’aspettativa tra i       │
│  giocatori stessi riguardo alla retribuzione, imponend

In [35]:
output = result.raw
output = output.replace('```markdown', '')
output = output.replace('```', '')

In [36]:
from IPython.display import display, Markdown
Markdown(output)


# L’Impennata dei Stipendi nel Calcio: Cosa Significa per i Giocatori e i Club?

## Introduzione
Negli ultimi anni, il calcio ha visto una continua e impressionante impennata dei stipendi dei giocatori, con cifre che adesso sembrano quasi extraterrestri. Questa crescita non è solo una questione di numeri; è fondamentale comprendere come queste retribuzioni influenzino le performance individuali degli atleti e la dinamica delle squadre. In questo articolo, esploreremo i trend attuali nei stipendi dei calciatori, come influiscono sulla motivazione e sul morale dei giocatori, e cosa possiamo aspettarci in futuro in questa economia sportiva in continua evoluzione.

Il tema degli stipendi nel calcio è di particolare rilevanza, non solo per i tifosi, ma anche per coloro che sono coinvolti nelle decisioni strategiche dei club. Comprendere la situazione salariale attuale aiuta non solo a contestualizzare le cifre, ma anche a valutare le implicazioni più ampie sui club e sulle rispettive performance. I salari dei calciatori, infatti, possiedono un forte impatto non solo sulla loro carriera, ma anche sulla visibilità e sulla competitività di una lega.

## 1. Recent Trends in Football Salaries
Iniziamo analizzando i salari medi dei calciatori nelle leghe maggiori come la Serie A, la Premier League e La Liga. Ad esempio, la Premier League continua a dominare con stipendi medi di oltre 3 milioni di euro all’anno, mentre la Serie A e La Liga stanno gradualmente recuperando terreno. Questo aumento esponenziale è in gran parte attribuibile a un’esplosione dei diritti televisivi e degli sponsor, che hanno ricolmato le casse dei club con liquidità significativa.

Oltre ai diritti di trasmissione, l'interesse globale per il calcio ha portato a una crescita senza precedenti nel merchandising e nelle vendite di biglietti, contribuendo a un circolo virtuoso di investimenti e guadagni. Queste forze di mercato hanno reso i giocatori d'élite più desiderabili, giustificando le cifre stratosferiche che vediamo oggi nei loro contratti. Questo fenomeno ha creato anche un’aspettativa tra i giocatori stessi riguardo alla retribuzione, imponendo ai club di essere competitivi non solo in campo, ma anche sul fronte economico.

## 2. Impact of High Salaries on Player Performance
Ma quali sono le reali implicazioni di questi stipendi alti sulle performance dei giocatori? Da un lato, una paga consistente può aumentare la motivazione degli atleti a dare il massimo in campo, mentre dall'altro c'è il rischio che la pressione di tali aspettative possa portare a prestazioni negative. La storia è ricca di esempi di atleti che, dopo aver ricevuto contratti enormi, hanno visto la loro capacità di giocare deteriorarsi.

Prendiamo il caso di alcuni giocatori che, dopo aver firmato nuovi contratti miliardari, hanno manifestato problemi di forma fisica o di concentrazione. Questo dimostra che, mentre il denaro può essere un potente motivatore, la gestione della pressione e le aspettative pubbliche possono essere una parte critica nel mantenere alte le performance. È importante pertanto che i club non solo incentivino i risultati, ma investano anche nel benessere psicologico e fisico dei loro giocatori.

## 3. Comparisons between Leagues
Esaminando i stipendi tra diverse leghe, emergono significative disparità. Mentre la Premier League offre i contratti più alti, altre leghe come la Serie A e la Bundesliga stanno progressivamente avvicinandosi, creando un terrain competitivo interessante. Tuttavia, queste differenze salariali possono influenzare le operazioni di mercato e le trasferte, lasciando alcuni club in situazione di svantaggio quando lottano per attrarre talenti di alto profilo.

Le conseguenze finanziarie di questa disparità si riflettono anche nella competitività generale delle squadre, influenzando la qualità e la visibilità globale delle leghe. Per esempio, un attraente stipendio potrebbe rendere un club più appetibile per i giocatori di talento, mentre i club con risorse limitate possono trovarsi a fare i conti con la difficoltà di mantenere i propri migliori elementi. Questa situazione richiede ai manager di considerare non solo i dati economici ma anche le implicazioni a lungo termine per la sostenibilità della propria squadra.

## 4. Future Projections
Guardando al futuro, le proiezioni sui stipendi dei calciatori nel prossimo decennio suggeriscono un’ulteriore crescita sostenuta. Con l'innovazione negli sport digitali e nuovi modelli di business che emergono in risposta ai cambiamenti economici, potremmo vedere nuovi flussi di introiti che influenzano ulteriormente l’allocazione dei fondi per i contratti. Tuttavia, è probabile che fattori esterni come crisi economiche, cambiamenti nelle preferenze dei tifosi e la sostenibilità saranno determinanti nel modellare il panorama economico del calcio.

La questione di come i club e i giocatori gestiranno questa liquidità e come queste scelte influenzeranno la lega rafforzerà l'importanza della pianificazione strategica. Gli scenari futuri potrebbero quindi portare anche a una rivalutazione delle strutture salariali e dei contratti degli atleti. La capacità di adattarsi a queste tendenze economiche emergerà come un fattore cruciale per il successo sia dei giocatori che dei club.

## Conclusione
Riflettendo sui trend attuali e le dinamiche in evoluzione intorno ai stipendi dei giocatori di calcio, è chiaro che la questione va ben oltre il semplice costo finanziario. Comprendere come questi stipendi influenzino le prestazioni, le trasferte e la competitività è fondamentale per chiunque sia coinvolto nel mondo del calcio. Invitiamo i lettori a sintonizzarsi su queste dinamiche economiche e a condividere le loro opinioni sui stipendi dei calciatori nei commenti sotto. Approfondite anche altri articoli sulla nostra piattaforma che esplorano ulteriormente l'economia del calcio.


# AI Agent to support aggregation of info from websites

Scrapping tool

In [ ]:
from crewai_tools.tools import ScrapeWebsiteTool

scrape_tool = ScrapeWebsiteTool()
scrape_tool = ScrapeWebsiteTool(website_url='https://www.repubblica.it/')

text = scrape_tool.run()
print(text[:500])

Sampple of Custom tool
*   Agent level: Agent can use the tool on any task performed
*   Task level: Agent can use the tool when performing that specific task (overrides previous one)

In [40]:
from langchain.tools import tool

@tool("Calculator")
def calculate(operation):
  """
    Useful to perform any math calculations, like sum, minus, multiplication and division.
    The input to this tool should be a mathematical expression, a coouple of example are `200*7` or `50/2`
  """

  try:
    return eval(operation)
  except Exception as e:
    return "Error: Invalid syntax in mathematical expression"

##Creating Agents with relative Tasks

In [57]:
newspaper_reader_agent = Agent(
    role="NewsPaperReader",
    goal="Collect news from all websites and summarize them",
    backstory=(
        "You work for an important communication company and you are asked to read all news from relevant websites. "
        "You should provide clear and concise summary of all the news. Use language Italian."
    ),
    allow_delegation=False,
    verbose=True
)

In [58]:
newspapaer_reading = Task(
    description=(
        "Read all the news from the following websites: {websites}\n"
        "Summarize the news in a bullet point list\n"
    ),
    expected_output="A bullet point list of the main news from websites.",
    tools=[scrape_tool],
    agent=newspaper_reader_agent
)

In [59]:
newspaper_filter_agent = Agent(
    role="NewsPaperFilter",
    goal="Extract only news related to specific topics",
    backstory=(
        "You work for an important communication company and you are asked to find all news from all relevant websites "
        "that are related to a specific topics. Use language Italian."
    ),
    allow_delegation=False,
    verbose=True
)

In [60]:
newspapaer_filter = Task(
    description=(
        "Read news summary and extract only news related to the following topics: {topics}"
        "Summarize the news in a bullet point list\n"
    ),
    expected_output="A bullet point list of the main news for each provided topic.",
    agent=newspaper_filter_agent
)

In [61]:
crew = Crew(
    agents=[newspaper_reader_agent, newspaper_filter_agent],
    tasks=[newspapaer_reading, newspapaer_filter],
    verbose=True ,
    memory=True
)

In [65]:
inputs = {
    "websites": ["https:www.repubblica.it/", "https:www.ilsole24ore.com/"],
    "topics": ["Economia", "Attualità"]
}

result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1534179b-6cf4-4600-8015-02a714268e9f                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:112: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  self.msg_id = ip.kernel._parent_header['header']['msg_id']

╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Historical Data:                                                                                               │
│  - Ensure the URLs are formatted correctly to prevent errors in data fetching.                                  │
│  - Include a clear objective in the task description to outline what specific types of news to summarize        │
│  (e.g., politics, economy).                                                                                     │
│  - Consider specifying the language for the summaries, as the expected output may differ based on language      │
│  content.                                                                                                       │
│  - Instruct to provide sources or links along with the news bullet points for verification and further          │
│  reading.                                                                                                       │
│  Recent Insights:                                                                                               │
│  - ```                                                                                                          │
│  Thought: I now know th...                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────── Retrieval Time: 567.48ms ────────────────────────────────────────────╯

...

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:112: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  self.msg_id = ip.kernel._parent_header['header']['msg_id']

/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 
'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model 
class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 63946477-dac9-4da9-b602-24d1c054dd54                                                                     │
│  Agent: NewsPaperFilter                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1534179b-6cf4-4600-8015-02a714268e9f                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: - Economia:                                                                                      │
│    - Viene ripristinato il bonus rifiuti con uno sconto del 25% sulla Tari.                                     │
│                                                                                                                 │
│  - Attualità:                                                                                                   │
│    - Due neonati sono morti in un ospedale di Bolzano a causa di un'infezione batterica.                        │
│    - È stato definito il programma del vertice Trump-Putin, che prevede una conferenza stampa congiunta.        │
│    - Trump vuole proporre un accordo con la Russia riguardo alle terre rare dell'Alaska.                        │
│    - La deputata Quartapelle ha commentato che Meloni non è considerata una leader dell'Europa.                 │
│    - Il reportage da Odessa racconta la vita sotto il costante timore della guerra.                             │
│    - A La Spezia, un uomo ha ucciso la sua ex moglie, con indagini sulla funzionalità del braccialetto          │
│  elettronico.                                                                                                   │
│    - Il sindaco Decaro ha dichiarato che non si candiderà alle regionali se Emiliano e Vendola decideranno di   │
│  presentarsi.                                                                                                   │
│    - Ci sono polemiche riguardo alla gestione dei migranti e ai centri di accoglienza.                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [67]:
output = result.raw

In [68]:
from IPython.display import display, Markdown
Markdown(output)

- Economia:
  - Viene ripristinato il bonus rifiuti con uno sconto del 25% sulla Tari.

- Attualità:
  - Due neonati sono morti in un ospedale di Bolzano a causa di un'infezione batterica.
  - È stato definito il programma del vertice Trump-Putin, che prevede una conferenza stampa congiunta.
  - Trump vuole proporre un accordo con la Russia riguardo alle terre rare dell'Alaska.
  - La deputata Quartapelle ha commentato che Meloni non è considerata una leader dell'Europa.
  - Il reportage da Odessa racconta la vita sotto il costante timore della guerra.
  - A La Spezia, un uomo ha ucciso la sua ex moglie, con indagini sulla funzionalità del braccialetto elettronico.
  - Il sindaco Decaro ha dichiarato che non si candiderà alle regionali se Emiliano e Vendola decideranno di presentarsi.
  - Ci sono polemiche riguardo alla gestione dei migranti e ai centri di accoglienza.